In [8]:
!pip install boto3 openai

In [16]:
# Brand new version to include extract full text content and then generate QnA with saving the excel to S3, also
# Import essential libraries
import pandas as pd
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import csv
import os
import openai
import boto3
from io import BytesIO
from io import StringIO
import xml.etree.ElementTree as ET
from time import sleep

# OpenAI Settings
# Organization Setting
openai.organization = ""

#OpenAI API Key
openai.api_key = ''

#data_df = pd.DataFrame(columns=['Title','DOI','Authors','Full_Text_URL','Full_Text_Content'])

# Method to generate QnA
def get_response(text):
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt="Generate 2 analysis or why type of questions and answers on  the following paragraph: "+text,
      temperature=0.9,
      max_tokens=160,
      top_p=1,
      frequency_penalty=1,
      presence_penalty=1
    )

    r = response.choices[0].text
    return r

def springernature_api(keywords):
  #API URL
  api_url = "http://api.springernature.com/"

  #API Key
  api_key = "523d9701c994f13cd90286c9078f295a"

  # Collections - (1) meta (2) metadata (3) openaccess
  collection = "openaccess"

  # Result Format - pam, jats, json, jsonp
  result_format = "json"

  # max result - 10 results if "p" parameter is not included
  q= "keyword:" + keywords
  p=100 # max is 100

  url = api_url + collection + "/" + result_format + "?api_key=" + api_key + "&q=" + q +"&p=100"

  response = requests.get(url)
  json_response = response.json()

  # Retrieve the number and pages of records
  no_of_record = json_response["result"][0]["total"]
  no_of_pages = int(no_of_record)//100
  if int(no_of_record)%100!=0:
    no_of_pages+=1

  url_list_in_pages = []

  for pagenum in range(no_of_pages):
    s_value = (pagenum*100)+1
    url_page = url + "&s=" + str(s_value)
    url_list_in_pages.append(url_page)


  # retrieve resposne from all the pages and urls
  for url_list_in_each_page in url_list_in_pages:
    each_page_response = requests.get(url_list_in_each_page)
    each_page_json_response = each_page_response.json()
    each_page_articles = each_page_json_response["records"]
    each_page_data_df = pd.DataFrame(columns=['Title','DOI','Authors','Full_Text_URL','Full_Text_Content'])
    for each_page_article in each_page_articles:
        each_page_title = each_page_article.get("title")
        each_page_doi = each_page_article.get("doi")
        each_page_creators = each_page_article.get("creators")
        each_page_authors = []
        for each_page_author in each_page_creators:
          each_page_authors.append(each_page_author['creator'])

        each_page_full_text_url = each_page_article.get("url")
        each_page_full_text_url = each_page_full_text_url[0]['value']


        each_page_full_text_response = requests.get(each_page_full_text_url)
        soup = BeautifulSoup(each_page_full_text_response.content, 'html.parser')
        each_page_full_text_content = soup.find('article').get_text() if soup.find("article") else 'no element available'

        # Insert the data into data_df DataFrame
        each_page_data_df.loc[len(each_page_data_df)] = [each_page_title,each_page_doi,each_page_authors,each_page_full_text_url, each_page_full_text_content]
    sleep(0.3)

  return each_page_data_df


def solr_api(keywords, api_provider):

  keywords_string = '+'.join(keywords)

  if api_provider == "wellcome":
        api_url = "https://wellcomeopenresearch.org/extapi/search?q=R_FT:" + keywords_string + "&wt=json"
  elif api_provider == "gatesopen":
        api_url = "https://gatesopenresearch.org/extapi/search?q=R_FT:" + keywords_string + "&wt=json"
  elif api_provider == "f1000":
        api_url = "https://f1000research.com/extapi/search?q=R_FT:" + keywords_string + "&wt=json"


  # Obtain the response and retrieve the total number of pages
  response = requests.get(api_url)
  data = response.content.decode()
  print(f"data: {data}")
  root = ET.fromstring(data)
  total_pages = root.get('totalNumberOfPages')
  print("Total number of pages: " + total_pages)


  # Extract all DOI items in each page and put to the list - doi_list
  doi_list = []
  downloaded_dois = set()

#This code transfers the content of the doi file and stores in the downloaded_dois set
  if os.path.exists("downloaded_dois.txt"):
    with open("downloaded_dois.txt", "r") as file:
        for line in file:
            downloaded_dois.add(line.strip())

  doi_items = []
  for page in range(int(total_pages)):
    url = api_url + "&page=" + str(page+1)
    response_page = requests.get(url)
    page_data = response_page.content.decode()
    page_root = ET.fromstring(page_data)
    doi_items.extend(page_root.findall('doi'))


#this stores the doi names in doi variable then adds only the dois not contained in our downloaded_dois set to the downloaded_dois set. hence duplicates cannot be added and then add these unique dois in the doi_list as well.
  for doi_element in doi_items:
        doi = doi_element.text
        if doi not in downloaded_dois:
            downloaded_dois.add(doi)
            doi_list.append(doi)

  # Update the persistent data store file with the newly downloaded DOIs...in othwe words, it adds the dois in the doi_list to the downloaded_dois.txt file
  with open("downloaded_dois.txt", "a") as file:
      for doi in doi_list:
          file.write(doi + "\n")
          print ("doi downloaded")

  data_df = pd.DataFrame(columns=['Title','DOI','Authors','Full_Text_URL','Full_Text_Content'])

  # Extract the full text content from each DOI link
  for doi_list_item in doi_list:

    if api_provider=="wellcome":
      doi_xml_link = "https://wellcomeopenresearch.org/extapi/article/xml?doi="
    elif api_provider=="gatesopen":
      doi_xml_link = "https://gatesopenresearch.org/extapi/article/xml?doi="
    elif api_provider=="f1000":
      doi_xml_link = "https://f1000research.com/extapi/article/xml?doi="

    doi_url = doi_xml_link + doi_list_item

    doi_page_response = requests.get(doi_url)

    if doi_page_response.status_code == 200:
      soup = BeautifulSoup(doi_page_response.content, 'html.parser')

      full_text_content_element = soup.find('body')
      full_text_content = full_text_content_element.get_text() if full_text_content_element else 'Full text content not found'

      title_element = soup.find("article-title")
      title = title_element.text if title_element else 'Title not found'
      print(title)

      # Retrieve authors list
      authors = []
      contribs = soup.find('front').find_all('contrib', {'contrib-type': 'author'}) if soup.find('front') else 'no element available'
      # looping through each author element and retrieving surname and given-names
      for contrib in contribs:
        surname_element = contrib.find('surname')
        surname = surname_element.text if surname_element else 'Surname not found'

        given_names_element = contrib.find('given-names')
        given_names = given_names_element.text if given_names_element else 'Given names not found'


        authors.append({surname, given_names})

      doi = doi_list_item
      full_text_url = doi_url

      data_df.loc[len(data_df)] = [title,doi,authors,full_text_url, full_text_content]

    else:
      title="NA"
      doi = doi_list_item
      authors="NA"
      full_text_url = doi_url
      full_text_content="NA"
      data_df.loc[len(data_df)] = [title,doi,authors,full_text_url, full_text_content]

  print(data_df)
  return data_df



# Method to ingress data via API
# api_provider - Springer nature (springnature), Gates Open Research (gateopen), Wellcome Open Research (wellcome), F1000 Research (f1000)
def data_ingression(keywords, api_provider, filename_output):
  #provides = api_provider


  if api_provider == "springer":
        data_df = springernature_api(keywords)
        print(f"data_df after calling springernature_api: {data_df}")
  elif api_provider == "gatesopen":
        data_df = solr_api(keywords, api_provider)
        print(f"data_df after calling solr_api with gatesopen: {data_df}")
  elif api_provider == "wellcome":
        data_df = solr_api(keywords, api_provider)
        print(f"data_df after calling solr_api with wellcome: {data_df}")
  elif api_provider == "f1000":
        data_df = solr_api(keywords, api_provider)
        print(f"data_df after calling solr_api with f1000: {data_df}")
  else:
        raise ValueError(f"Invalid api_provider: {api_provider}")


  # Save the DataFrame as a excel file to a temporary buffer
  excel_buffer = BytesIO()
  data_df.to_excel(excel_buffer, index=False)

  # Save the DataFrame as a excel file to a temporary buffer

  # S3 Client - Initialization and store the exported the excel file to the S3 Bucket

  # TiiQu's key
  s3_client = boto3.client('s3', aws_access_key_id='',
                          aws_secret_access_key='')


  s3_client.put_object(Bucket='tiiqu-openresearch', Key="data-ingression/" + filename_output, Body=excel_buffer.getvalue())

  #Convert excel to CSV
  #file_obj = s3_client.get_object(Bucket='nhdata2023', \\extapi\\article\\xmlKey="data-ingression/" + filename_output)['Body'].read()
  #excel_data = pd.read_excel(file_obj)
  #csv_buffer = StringIO()
 # excel_data.to_csv(csv_buffer, index=False)
 # s3_client.put_object(Body=csv_buffer.getvalue(), Bucket='nhdata2023', Key="data-ingression/" + filename_output.replace(".xlsx",".csv"))



In [23]:
import datetime


# List of keywords
keywords_list = ['Sustainable+energy', 'Clean+energy', 'Green+energy', 'Pollution+regulations', 'Pollution+mitigation', 'Pollutants', 'Paris+Agreement', 'Fossil+fuels', 'Deforestation']

# API providers
api_providers = ['f1000', 'wellcome', 'gatesopen']  # Choose the desired API providers

# Get the current date
current_date = datetime.datetime.now().strftime('%Y%m%d')

# Initialize the counter variable and set the maximum number of iterations
count = 0
max_iterations = len(keywords_list)

# Loop until the maximum number of iterations is reached or all keyword combinations have been processed
while count < max_iterations:
    keyword = keywords_list[count]
    print("Current count:", count + 1)

    # Iterate through each API provider
    for api_provider in api_providers:
        # Generate the filename_output
        filename_output = keyword + '_' + api_provider + '_' + current_date + '.xlsx'
        print("Filename output:", filename_output + "\n")


        # Call the data_ingression function
        data_ingression(keyword, api_provider, filename_output)

    # Increment the counter
    count += 1

# Print the total number of times data_ingression function was called
print("Total data_ingression function calls:", count)


Current count: 1
Filename output: Sustainable+energy_f1000_20230717.xlsx

data: <?xml version="1.0" encoding="UTF-8" standalone="yes"?><results numberOfResultsInPage="100" totalNumberOfPages="2"><doi>10.12688/f1000research.2-204.v1</doi><doi>10.12688/f1000research.3-47.v2</doi><doi>10.12688/f1000research.4436.2</doi><doi>10.12688/f1000research.5530.2</doi><doi>10.12688/f1000research.5934.1</doi><doi>10.12688/f1000research.4583.2</doi><doi>10.12688/f1000research.5369.2</doi><doi>10.12688/f1000research.5654.2</doi><doi>10.12688/f1000research.6392.1</doi><doi>10.12688/f1000research.6536.1</doi><doi>10.12688/f1000research.2-71.v3</doi><doi>10.12688/f1000research.5921.2</doi><doi>10.12688/f1000research.7201.1</doi><doi>10.12688/f1000research.7287.1</doi><doi>10.12688/f1000research.7931.1</doi><doi>10.12688/f1000research.7507.2</doi><doi>10.12688/f1000research.7955.2</doi><doi>10.12688/f1000research.7694.2</doi><doi>10.12688/f1000research.8987.2</doi><doi>10.12688/f1000research.9434.1</doi><

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Determining the 
                    l
                        x
 and estimating the force of mortality for children under five in Bangladesh
ALL-IN meta-analysis: breathing life into living systematic reviews


/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Epidemiological evidence relating environmental smoke to COPD in lifelong non-smokers: a systematic review
                                               Title  \
0  Determining the \n                    l\n     ...   
1  ALL-IN meta-analysis: breathing life into livi...   
2  Epidemiological evidence relating environmenta...   

                              DOI  \
0  10.12688/f1000research.21202.1   
1  10.12688/f1000research.74223.1   
2  10.12688/f1000research.13887.3   

                                             Authors  \
0  [{Md. Rafiqul, Islam}, {Rana, Md. Sohel}, {Hos...   
1          [{ter Schure, Judith}, {Grünwald, Peter}]   
2  [{Lee, Peter N.}, {Barbara A., Forey}, {Kathar...   

                                       Full_Text_URL  \
0  https://f1000research.com/extapi/article/xml?d...   
1  https://f1000research.com/extapi/article/xml?d...   
2  https://f1000research.com/extapi/article/xml?d...   

                                   Full_Text_Content  
0  \n\nAbbrevi

...

Impact of antiretroviral therapy regimens adherence on perceived health and wellbeing status among adults living with HIV in Ghana


/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Neuraxial opioids as analgesia in labour, caesarean section and hysterectomy: A questionnaire survey in Sweden
                                               Title  \
0     Multiple statistical tests: Lessons from a d20   
1  Physiological and biological characterization ...   
2  Health related quality of life in COVID-19 sur...   
3  Disulfiram repurposing combined with nutrition...   
4  Development, validation and reliability of a q...   
5  Inequality and migration in Kenya: Investigati...   
6  Impact of antiretroviral therapy regimens adhe...   
7  Neuraxial opioids as analgesia in labour, caes...   

                              DOI  \
0   10.12688/f1000research.8834.2   
1  10.12688/f1000research.11698.2   
2  10.12688/f1000research.50781.1   
3  10.12688/f1000research.16786.1   
4  10.12688/f1000research.52997.2   
5  10.12688/f1000research.74058.1   
6  10.12688/f1000research.75374.1   
7  10.12688/f1000research.10705.2   

                                             Autho

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


A cross-sectional analysis identifies a low prevalence of 
                    Plasmodium ovale
curtisi infections in symptomatic and asymptomatic individuals in Kilifi county, Kenya.
Impact of the first COVID-19 pandemic wave on the Scottish Multiple Sclerosis Register population
                                               Title  \
0  A cross-sectional analysis identifies a low pr...   
1  Impact of the first COVID-19 pandemic wave on ...   

                                DOI  \
0  10.12688/wellcomeopenres.17972.1   
1  10.12688/wellcomeopenres.16349.1   

                                             Authors  \
0  [{Akinyi, Mercy Yvonne}, {Chifwete, Margaret},...   
1  [{Fernandes, Peter M.}, {Martin, O'Neill}, {Ke...   

                                       Full_Text_URL  \
0  https://wellcomeopenresearch.org/extapi/articl...   
1  https://wellcomeopenresearch.org/extapi/articl...   

                                   Full_Text_Content  
0  \n\nIntroduction\nMalaria is one of

In [4]:
import os

# Get the list of files and directories in the current directory
file_list = os.listdir('/content')

# Print the list of files and directories
for file in file_list:
    print(file)

.config
downloaded_dois.txt
sample_data


In [5]:
import shutil
import os

#clear the content of the /content directory in google colab
folder_path = '/content'
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        os.remove(file_path)
    elif os.path.isdir(file_path):
        shutil.rmtree(file_path)

In [7]:
import os

# Get the list of files and directories in the current directory
file_list = os.listdir('/content')

# Print the list of files and directories
for file in file_list:
    print(file)